In [1]:
from IPython.core.display import display, HTML

from config import ERC721_ANALYSIS_DATASET_NAME, AZIMUTH_POINTS_TABLE_NAME, SNAPSHOT_TS
from src.utils_bigquery import get_df, create_table_from_df
from src.extractor_urbit import extract_point_data

SELECT_TRANSFERRED_TOKENS = True
DROP_TABLES = True
CREATE_TABLES = True

AZIMUTH_CONTRACT_ADDRESS = '0x223c067F8CF28ae173EE5CafEa60cA44C335fecB'.lower()
LINEAR_STAR_RELEASE_CONTRACT_ADDRESS= '0x86cd9cd0992F04231751E3761De45cEceA5d1801'.lower()
SPAWNED_EVENT_TOPIC = '0xb2d3a6e7a339f5c8ff96265e2f03a010a8541070f3744a247090964415081546'
ZERO_ADDRESS = '0x0000000000000000000000000000000000000000'
AZIMUTH_CONTRACT_CREATE_TS = '2018-11-27 00:00:00'

## Get List of Transferred Tokens

In [2]:
if SELECT_TRANSFERRED_TOKENS:
    query_1 = f'''
    SELECT cast(topics[ORDINAL(3)] AS INT64) as point
    FROM `bigquery-public-data.crypto_ethereum.logs`
    WHERE block_timestamp > '{AZIMUTH_CONTRACT_CREATE_TS}'
      AND block_timestamp < '{SNAPSHOT_TS}'
      AND address = '{AZIMUTH_CONTRACT_ADDRESS}'
      AND topics[ORDINAL(1)] = '{SPAWNED_EVENT_TOPIC}'
    '''

    azimuth_tokens_list = list(get_df(query=query_1).point)
    print(f'Number of Azimuth Tokens: {len(azimuth_tokens_list):>,}')

Number of Azimuth Tokens: 75,704


## Get Point Owners and Types

In [3]:
points_df = extract_point_data(point_list=azimuth_tokens_list, pool_size=10, chunk_size=64)

Processing 75704 points split into 1183 chunks on 10 processes:
Points 24544:52655 extracted
Points 3743352064:1383937440 extracted
Points 22661:47507 extracted
Points 14052:7610 extracted
Points 14675:12880 extracted
Points 56630:23365 extracted
Points 54241:37744 extracted
Points 35043:36791 extracted
Points 4117038336:3530 extracted
Points 27769:19888 extracted
Points 38000:58601 extracted
Points 47763:64628 extracted
Points 14160:60929 extracted
Points 20144:39641 extracted
Points 23621:22865 extracted
Points 3786:27812 extracted
Points 52911:26611 extractedPoints 1664496820:20051 extracted

Points 37047:6094 extracted
Points 7866:8807 extracted
Points 58857:21196 extracted
Points 61185:65067 extracted
Points 23121:62547 extracted
Points 64884:23993 extracted
Points 6350:49524 extracted
Points 55675:7900 extracted
Points 28068:39298 extracted
Points 20307:59187 extracted
Points 39897:46762 extracted
Points 26867:14041 extracted
Points 18636:40560 extracted
Points 45366:30017 extrac

In [4]:
display(HTML(points_df.head().to_html(index=False, notebook=True, show_dimensions=False)))

point,point_size,point_type,owner,parent_point
3743352064,2,planet,0x797dc75b60cb63984c1221345d0c7b549cb7f978,1280
300090624,2,planet,0x797dc75b60cb63984c1221345d0c7b549cb7f978,1280
3768582578,2,planet,0x3b11e293637a710f114a86d4b27ad3bac282238d,434
61552032,2,planet,0xb9056e52458d745aacc0a06124e26567d7319726,13728
2587346,2,planet,0x7bae5d327007ad1b5e74a455396acaef7ba6c995,31442


## Insert Point Data

In [5]:
create_table_from_df(source_df=points_df,
                     dataset_name=ERC721_ANALYSIS_DATASET_NAME,
                     table_name=AZIMUTH_POINTS_TABLE_NAME,
                     drop_existing_table=DROP_TABLES)

Table erc721_analysis:azimuth_points has been deleted.
Table erc721_analysis:azimuth_points has been created.


True